In [24]:
#@markdown Run to install repo
import os
import sys

repo_url = "https://github.com/paulynamagana/AFDB_scripts"

repo_name = repo_url.split("/")[-1]  # Extract the repository name from the URL (after the last "/")

def find_repository(repo_name):
    """Searches the entire file system for a directory with the given name."""
    for root, dirs, files in os.walk("/"):  # Start from the root directory
        if repo_name in dirs:
            return os.path.join(root, repo_name)  # Return the full path
    return None

repo_path = find_repository(repo_name)

if repo_path:
    print(f"Found repository {repo_name} at: {repo_path}")
    %cd {repo_path}

    # (Optional) Update the repository
    !git pull  # Pull latest changes (if any)

else:
    print(f"Repository {repo_name} not found. Cloning...")
    !git clone {repo_url}
    %cd {repo_name}


sys.path.append(os.path.join("/content/", repo_name))

KeyboardInterrupt: 

In [1]:
from api_handler import fetch_AFDB_data, extract_alpha_missense_url, extract_pdb_url, extract_pae_url

uniprot_id = "P05067"
alphafold_data = fetch_AFDB_data(uniprot_id)
if alphafold_data:
    am_data_url = extract_alpha_missense_url(alphafold_data)
    pdb_data_url = extract_pdb_url(alphafold_data)
    pae_png_url = extract_pae_url(alphafold_data)



2024-09-12 21:27:29,718 - INFO - Fetching data for P05067
2024-09-12 21:27:30,067 - INFO - Successfully retrieved data for P05067
2024-09-12 21:27:30,069 - INFO - Retrieving AM data
2024-09-12 21:27:30,069 - INFO - Retrieving url for PDB file
2024-09-12 21:27:30,070 - INFO - Retrieving url for PAE png


In [9]:
from AM_data_processing import extract_am_data, calculate_average_pathogenicity, modify_pdb_with_am_data, plot_am_heatmap

am_data = extract_am_data(am_data_url)

if am_data is not None:
    average_scores = calculate_average_pathogenicity(am_data)
    modify_pdb_with_am_data(pdb_data_url, average_scores)
    plot_am_heatmap(am_data)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
#@markdown visualise structure
import requests
from IPython.display import Image, display
from plots import plot_plddt_legend
import matplotlib.pyplot as plt
from Bio.PDB import PDBParser

image_url_af = "https://alphafold.ebi.ac.uk/files/AF-P05067-F1-predicted_aligned_error_v4.png"

with open(f"data_output/{uniprot_id}_with_AM_scores.PDB", "r") as f:
  AM_pdb = f.read()

# Fetch and display the first structure
pdb_data = requests.get(pdb_data_url).text

plddt_colors = ["#FFFFFF", "#FF7D45", "#FFFF00", "#65CBF3", "#0000FF"]
# Define the gradient colors based on the image
color_positions = [
    (0.0, '#2166AC'),   # Blue
    (0.2, '#92C5DE'),   # Light blue
    (0.36, '#D1E5F0'),  # Very light blue
    (0.45, '#A8A9AC'),  # Grey
    (0.6, '#FDDDBC'),   # Light orange
    (0.8, '#F4A582'),   # Orange
    (1.0, '#B2182B')    # Red
]
color_gradient = [color for _, color in color_positions]


# Create the view and add models
view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js', width=690, height=500, viewergrid=(2, 1), linked=True)
view.addModel(pdb_data, 'pdb')
view.addModel(AM_pdb, 'pdb')
view.setStyle({'cartoon': {'colorscheme': {'prop': 'b', 'gradient': 'linear', 'min': 0, 'max': 125, "colors": plddt_colors }}}, viewer=(0,0))
# Set the style with the custom gradient
view.setStyle({'cartoon': {'colorscheme': {
    'prop': 'b',
    'gradient': 'linear',
    'min': 0,
    'max': 120,
    'colors': color_gradient
}}}, viewer=(1,0))

view.zoomTo()
view.show()


    # Create and display the plot
fig = plot_plddt_legend()
plt.show()  # Display the plot
plt.close(fig)  # Close the figure to free up memory


image_data1 = requests.get(image_url_af).content
display(Image(data=image_data1))


In [ ]:
from AM_data_processing import extract_pathogenicity_and_plddt

file_path= "data_output/AM_Scores_AF-P05067-F1-model_v4.pdb"
pathogenicity_scores, plddt_scores = extract_pathogenicity_and_plddt(file_path, pdb_data_url)

In [ ]:
from plots import plot_scores

plot_scores(pathogenicity_scores, plddt_scores)